# Qubit rotation

This example shows the basic operation of machine learning framework with a quantum device. A qubit is initilized with a arbitary Rx and Ry rotation and the target state is pure |1> state. After several steps of the iteration. The rotation angle of Rx and Ry will converge to 0 and pi. 

### About this example
The example contains the model compiled with three different configurations of backends and interfaces -- JAX backend, JAX backend with pytorch interface and pytorch backend.
The example also shows how to use state vector propagation mode and tensor network contraction mode. And two methods for obtaining gradient -- back propagation and parameter shift.

# Initialization

In [ ]:
import pennylane as qml

# Define the quantum model

### Define the circuit with TeD-Q framework
#### (Remember, if you have multiple measurements, all the measurement results should has the same shape!)

In [ ]:
n_qubits = 1
q_depth = 1

In [ ]:
dev = qml.device("default.qubit", wires=n_qubits)

In [ ]:
@qml.qnode(dev, interface="torch")
def circuitDef(params):
    for i in range(n_qubits):
        qml.RY(params[0][i], wires=i)
    for i in range(n_qubits):
        qml.RZ(params[1][i], wires=i)
    for i in range(n_qubits-1):
        qml.CNOT(wires=(i, i+1))
    for i in range(n_qubits):
        qml.RY(params[2][i], wires=i)
    for i in range(n_qubits):
        qml.RZ(params[3][i], wires=i)
    return [qml.expval(qml.PauliZ(wires=idx)) for idx in range(n_qubits)]


### Define cost function and optimizer

In [ ]:
def cost(params):
    results = circuitDef(params)
    result = sum(results)
    return result

In [ ]:
import torch
#parameters = torch.rand(((q_depth+1)*2,n_qubits), requires_grad= True)

In [ ]:
#a = cost(parameters)

In [ ]:
#a

In [ ]:
#a.backward()

In [ ]:
#parameters.grad

### Using backend's optimizer and training

In [ ]:
%%time
parameters = torch.rand(((q_depth+1)*2,n_qubits), requires_grad= True)
from torch import optim
optimizer = optim.Adam([parameters], lr=0.1)
for i in range(500):
    optimizer.zero_grad()
    #print(b.grad)
    loss = cost(parameters)
    loss.backward()
    optimizer.step()
    #if (i + 1) % 5 == 0:
    #    print("Cost after step {:5d}: {}".format(i + 1, cost(parameters)))
    #    print("Parameters after step {:5d}: {}".format(i + 1, parameters))
#print(parameters)
#print(cost(parameters))

In [1]:
import numpy as np
from qiskit import *
import torch
import math
from torch.optim import lr_scheduler
from qiskit.providers.aer import AerSimulator
from qiskit.circuit import Parameter
from qiskit_machine_learning.neural_networks import CircuitQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.utils import QuantumInstance, algorithm_globals

In [43]:
n_qubits = 4
q_depth = 1

In [44]:

n_train_params = (q_depth+1)*2*n_qubits
params_list = []
for idx in range(n_train_params):
    params_list.append(Parameter("param_"+str(idx)))


In [45]:
n_train_params

16

In [46]:
#     params = params.view(-1)
qnn_circ = QuantumCircuit(n_qubits, 0)
    

# Hd td
count = 0
for i in range(n_qubits):
    qnn_circ.ry(params_list[count],i)
    count += 1
for i in range(n_qubits):
    qnn_circ.rz(params_list[count],i)
    count += 1
for i in range(n_qubits-1):
    qnn_circ.cx(i, i+1)
for i in range(n_qubits):
    qnn_circ.ry(params_list[count],i)
    count += 1
for i in range(n_qubits):
    qnn_circ.rz(params_list[count],i)
    count += 1
print("count:  ", count)
    
   
qc = QuantumCircuit(n_qubits, 0)
qc.append(qnn_circ, range(n_qubits))
#qc.measure(0,0)

count:   16


In [47]:
import numpy as np

from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.opflow import StateFn, PauliSumOp, AerPauliExpectation, ListOp, Gradient, PauliOp, Zero
from qiskit.utils import QuantumInstance, algorithm_globals

algorithm_globals.random_seed = 42

from qiskit_machine_learning.neural_networks import OpflowQNN
from qiskit import quantum_info

In [48]:
qi = QuantumInstance(Aer.get_backend("aer_simulator_statevector"))
preparation_circ = QuantumCircuit(n_qubits, 0)

In [49]:
# set method to calculcate expected values
expval = AerPauliExpectation()

# define gradient method
gradient = Gradient()
qc_sfn1 = StateFn(qc)
p_s = ["Z" for _ in range(n_qubits)]
p_s = "".join(p_s)
H1 = StateFn(PauliOp(quantum_info.Pauli(p_s)))
op1 = ~H1 @ qc_sfn1
#qi = QuantumInstance(Aer.get_backend("aer_simulator_density_matrix"))
qnn2 = OpflowQNN(op1, preparation_circ.parameters, qnn_circ.parameters, expval, gradient, qi)

In [50]:
initial_weights = 0.1 * (2 * algorithm_globals.random.random(qnn2.num_weights) - 1)
print("Initial weights: ", initial_weights)

model2 = TorchConnector(qnn2, initial_weights)

Initial weights:  [ 0.05479121 -0.01222431  0.07171958  0.03947361 -0.08116453  0.09512447
  0.05222794  0.05721286 -0.07437727 -0.00992281 -0.0258404   0.085353
  0.02877302  0.06455232 -0.01131716 -0.05455226]


In [51]:
model2()

tensor([0.9952], grad_fn=<_TorchNNFunctionBackward>)

In [52]:
#optimizer = torch.optim.Adam(model2.parameters(), lr=0.5)

In [53]:
%%time
from torch import optim
optimizer = optim.Adam(model2.parameters(), lr=0.1)
for i in range(500):
    optimizer.zero_grad()
    #print(b.grad)
    loss = model2()
    loss.backward()
    optimizer.step()
    #if (i + 1) % 5 == 0:
    #    print("Cost after step {:5d}: {}".format(i + 1, cost(parameters)))
    #    print("Parameters after step {:5d}: {}".format(i + 1, parameters))
#print(parameters)
#print(cost(parameters))

CPU times: user 1min 54s, sys: 29.1 s, total: 2min 24s
Wall time: 2min 32s


In [82]:
import paddle
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import plot_state_in_bloch_sphere
import numpy as np
from paddle_quantum.utils import pauli_str_to_matrix
n_qubits = 7
q_depth = 1

In [83]:
def circuit(param):
    
    cir = UAnsatz(n_qubits)
    # 对单比特量子态做随机的旋转操作
    
    for i in range(n_qubits):
        cir.ry(param[0][i], i)
    for i in range(n_qubits):
        cir.rz(param[1][i], i)
    for i in range(n_qubits-1):
        cir.cnot([i, i+1])
    for i in range(n_qubits):
        cir.ry(param[2][i], i)
    for i in range(n_qubits):
        cir.rz(param[3][i], i)
    
    #cir.rx(param[0], 0)
    #cir.ry(param[1], 0)
    #cir.pauli_channel(1, 0, 0, 1)
    #cir.pauli_channel(0, 1, 0, 2)
    #cir.pauli_channel(0, 0, 1, 3)
    #cir.s(4)
    #cir.t(5)
    #cir.cnot([6, 7])
    #cir.cy([8, 9])
    #cir.cz([10, 11])
    #cir.swap([12, 13])
    #cir.cswap([14, 15, 16])
    #cir.ccx([17, 18, 19])
    
    cir.run_state_vector()
    
    return cir
    

In [84]:
[[i+1, 'Z0'] for i in range(n_qubits)]

[[1, 'Z0'], [2, 'Z0'], [3, 'Z0'], [4, 'Z0'], [5, 'Z0'], [6, 'Z0'], [7, 'Z0']]

In [85]:
def cost(param):
    cir = circuit(param)
    fin_state = cir.run_state_vector()
    H = [[1, 'Z0']]
    expectation_val = cir.expecval(H)
    loss = expectation_val
    return loss

In [86]:
a = paddle.to_tensor(np.random.rand((q_depth+1)*2, n_qubits), stop_gradient = False)
param = (a,)

In [87]:
a[3]

Tensor(shape=[7], dtype=float64, place=CPUPlace, stop_gradient=False,
       [0.61317793, 0.75366663, 0.78116963, 0.48335164, 0.47360398, 0.45203765,
        0.15946189])

In [88]:
loss = cost(*param)

In [89]:
opt = paddle.optimizer.SGD(learning_rate = 0.4, parameters = param) 

In [90]:
%%time
for i in range(500):
    loss = cost(*param)
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

CPU times: user 56.8 s, sys: 136 ms, total: 57 s
Wall time: 57 s


In [91]:
print("Optimized rotation angles: {}".format(param))

Optimized rotation angles: (Tensor(shape=[4, 7], dtype=float64, place=CPUPlace, stop_gradient=False,
       [[1.44618449, 1.57079633, 0.86073060, 0.29682761, 0.11007738, 0.44460832,
         0.65599896],
        [0.00000000, 0.00000000, 0.79914506, 0.69832185, 0.29803606, 0.56443939,
         0.31511345],
        [1.69540817, 0.37198606, 0.47648065, 0.62191210, 0.02060302, 0.99209230,
         0.66123391],
        [0.61317793, 0.75366663, 0.78116963, 0.48335164, 0.47360398, 0.45203765,
         0.15946189]]),)


In [92]:
print("Cost: {}".format(loss))

Cost: Tensor(shape=[1], dtype=float64, place=CPUPlace, stop_gradient=False,
       [-1.00000000])


# Qubit rotation

This example shows the basic operation of machine learning framework with a quantum device. A qubit is initilized with a arbitary Rx and Ry rotation and the target state is pure |1> state. After several steps of the iteration. The rotation angle of Rx and Ry will converge to 0 and pi. 

### About this example
The example contains the model compiled with three different configurations of backends and interfaces -- JAX backend, JAX backend with pytorch interface and pytorch backend.
The example also shows how to use state vector propagation mode and tensor network contraction mode. And two methods for obtaining gradient -- back propagation and parameter shift.

# Initialization

In [ ]:
import tedq as qai

# Define the quantum model

### Define the circuit with TeD-Q framework
#### (Remember, if you have multiple measurements, all the measurement results should has the same shape!)

In [ ]:
n_qubits = 10
q_depth = 1

In [ ]:
# Define quantum circuit
def circuitDef(params):
    qai.templates.HardwareEfficient(n_qubits, q_depth, params)
    return [qai.expval(qai.PauliZ(qubits=[idx])) for idx in range(n_qubits)]


parameter_shapes = [((q_depth+1)*2,n_qubits)]

# Quantum circuit construction
circuit = qai.Circuit(circuitDef, n_qubits, parameter_shapes = parameter_shapes)

In [ ]:
# visualization of the quantum circuit
drawer = qai.matplotlib_drawer(circuit)
drawer.draw_circuit()

# Circuit compiled with pytorch backend

Gradient will obtain from backpropagation by default

### state vector propagation mode

In [ ]:
my_compilecircuit = circuit.compilecircuit(backend="pytorch")

### Define cost function and optimizer

In [ ]:
def cost(*params):
    results = my_compilecircuit(*params)
    result = sum(results)
    return result

In [ ]:
#import torch
#from functorch import vmap
#parameters = torch.rand((5, (q_depth+1)*2,n_qubits), requires_grad= True)

In [ ]:
#vmap(cost)(parameters)

In [ ]:
#a = cost(parameters)

In [ ]:
#a

In [ ]:
#a.backward()

In [ ]:
#parameters.grad

### Using backend's optimizer and training

In [ ]:
%%time
parameters = torch.rand(((q_depth+1)*2,n_qubits), requires_grad= True)
from torch import optim
optimizer = optim.Adam([parameters], lr=0.1)
for i in range(500):
    optimizer.zero_grad()
    #print(b.grad)
    loss = cost(parameters)
    loss.backward()
    optimizer.step()
    #if (i + 1) % 5 == 0:
    #    print("Cost after step {:5d}: {}".format(i + 1, cost(parameters)))
    #    print("Parameters after step {:5d}: {}".format(i + 1, parameters))
#print(parameters)
#print(cost(parameters))